In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations

In [2]:
db_raw = pd.read_excel('salesCG201718.xls', header=None)

In [3]:
db_data_month = db_raw.drop([0,1], axis=0)
db_data_month['month'] = db_data_month[0].dt.month
sales_month = db_data_month.groupby('month').sum()

In [4]:
shops = db_raw.transpose()[[0,1]].drop([0], axis=0)
shops.rename(columns = {0:'shop', 1:'product'}, inplace=True)

for i in range(1,21,1):
    shops['shop'][2*i] = shops['shop'][2*i-1]

In [6]:
tr_sales_month = sales_month.transpose()
monthly_sales_per_shop = pd.concat([shops, tr_sales_month], axis=1).set_index(['shop', 'product']).transpose()

In [7]:
av_daily_per_month = monthly_sales_per_shop/(6*4)

In [8]:
rounded_av_daily_per_month = av_daily_per_month.apply(np.round)

In [11]:
for i in range(1,13,1):
    monthly_per_month = av_daily_per_month.transpose()[i]

In [12]:
for num_s in range(1,21,1):
    rounded_av_daily_per_month[('S'+str(num_s), 'P1')] = rounded_av_daily_per_month[('S'+str(num_s), 'P1')]*0.8
    rounded_av_daily_per_month[('S'+str(num_s), 'P2')] = rounded_av_daily_per_month[('S'+str(num_s), 'P2')]*0.4

In [13]:
vol_av_daily_per_month = rounded_av_daily_per_month.transpose().groupby('shop').sum().reindex(monthly_per_month.index.get_level_values(0)).drop_duplicates().copy()

In [14]:
vol_av_daily_per_month

1.0   2.0    3.0    4.0    5.0    6.0    7.0    8.0    9.0   10.0  11.0  \
shop                                                                            
S1    2.40  2.56   3.52   5.28   5.12   5.12   5.12   5.28   4.16  3.84  2.08   
S2    2.24  2.56   3.52   4.64   5.76   3.84   5.12   5.28   3.52  3.84  2.24   
S3    3.04  4.00   4.96   6.72   6.40   6.40   7.04   6.08   5.60  4.80  3.20   
S4    3.36  3.36   5.60   5.44   7.04   7.04   5.76   6.08   4.96  4.48  2.56   
S5    2.24  3.20   4.96   4.00   5.76   4.48   5.12   5.92   3.52  3.36  2.24   
S6    3.36  4.48   4.32   5.92   5.76   7.04   8.96   7.20   5.60  4.64  3.20   
S7    4.96  5.44   8.48   8.64   8.48   8.32   9.76   8.64   8.32  7.68  4.16   
S8    4.16  4.96   7.20   9.92  14.88  10.24  10.88   9.44   8.96  6.56  4.48   
S9    6.08  6.24   9.92  11.36  12.96  12.80  12.80   8.80   9.76  6.72  4.80   
S10   4.32  4.96   5.12   7.36  10.24   7.68   8.96   7.36   7.68  5.76  3.36   
S11   4.48  6.88   7.04   9.44   9.12  10.88  11.68   9.92   7.84  5.60  4.48   
S12   5.44  5.28   7.68   7.36  11.68   9.60  10.40  11.36   7.68  6.40  4.48   
S13   5.60  6.24   7.68   8.00  10.24  11.52  10.24   7.36   8.48  7.84  3.36   
S14   5.44  6.56   7.36  11.36  10.40  10.88  13.60   8.80   8.48  8.48  5.76   
S15   4.64  5.76   8.00   9.44  10.40  12.16   9.76   9.92   8.48  7.04  4.48   
S16   6.72  8.48  11.20  13.28  12.96  12.16  14.24  13.92  10.72  7.84  6.24   
S17   5.12  4.64   7.04   8.00   6.40   9.60   8.96   8.00   5.60  5.44  3.52   
S18   3.20  3.36   5.60   7.36   5.76   6.40   6.40   5.92   4.96  4.64  3.20   
S19   2.40  3.04   4.16   4.00   5.12   5.12   3.84   4.00   3.68  3.20  2.40   
S20   3.20  3.36   6.24   7.36   7.04   5.76   6.40   6.56   6.24  4.64  2.40   

      12.0  
shop        
S1    1.12  
S2    1.60  
S3    1.92  
S4    1.76  
S5    1.28  
S6    1.92  
S7    2.40  
S8    2.72  
S9    2.56  
S10   2.40  
S11   2.24  
S12   2.40  
S13   2.56  
S14   3.20  
S15   2.72  
S16   3.68  
S17   2.40  
S18   1.44  
S19   1.28  
S20   1.76

Index(['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11',
       'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20'],
      dtype='object', name='shop')

In [16]:
def rSubset(shops, r):
  
    # return list of all subsets of length r
    # to deal with duplicate subsets use 
    # set(list(combinations(shops, r)))
    return list(combinations(shops, r))

In [17]:
#all combinations of shops in one delivery round
if __name__ == "__main__":
    shops = vol_av_daily_per_month.index #all the shop names
    
    for r in range(0, 5): #the max amount of shops per set is 4
        for i in range(0,len(rSubset(shops, r))):
            vol_sum = 0
            for j in range(0,r):
                vol_sum = vol_sum + vol_av_daily_per_month[1][rSubset(shops, r)[i][j]] #value of shop 
        
            if vol_sum <= 20:
                print(rSubset(shops, r)[i])
                
#now we need to make sets of all combinations of combinations so that all shops are reached

()
('S1',)
('S2',)
('S3',)
('S4',)
('S5',)
('S6',)
('S7',)
('S8',)
('S9',)
('S10',)
('S11',)
('S12',)
('S13',)
('S14',)
('S15',)
('S16',)
('S17',)
('S18',)
('S19',)
('S20',)
('S1', 'S2')
('S1', 'S3')
('S1', 'S4')
('S1', 'S5')
('S1', 'S6')
('S1', 'S7')
('S1', 'S8')
('S1', 'S9')
('S1', 'S10')
('S1', 'S11')
('S1', 'S12')
('S1', 'S13')
('S1', 'S14')
('S1', 'S15')
('S1', 'S16')
('S1', 'S17')
('S1', 'S18')
('S1', 'S19')
('S1', 'S20')
('S2', 'S3')
('S2', 'S4')
('S2', 'S5')
('S2', 'S6')
('S2', 'S7')
('S2', 'S8')
('S2', 'S9')
('S2', 'S10')
('S2', 'S11')
('S2', 'S12')
('S2', 'S13')
('S2', 'S14')
('S2', 'S15')
('S2', 'S16')
('S2', 'S17')
('S2', 'S18')
('S2', 'S19')
('S2', 'S20')
('S3', 'S4')
('S3', 'S5')
('S3', 'S6')
('S3', 'S7')
('S3', 'S8')
('S3', 'S9')
('S3', 'S10')
('S3', 'S11')
('S3', 'S12')
('S3', 'S13')
('S3', 'S14')
('S3', 'S15')
('S3', 'S16')
('S3', 'S17')
('S3', 'S18')
('S3', 'S19')
('S3', 'S20')
('S4', 'S5')
('S4', 'S6')
('S4', 'S7')
('S4', 'S8')
('S4', 'S9')
('S4', 'S10')
('S4', 'S11'

('S3', 'S10', 'S20')
('S3', 'S11', 'S12')
('S3', 'S11', 'S13')
('S3', 'S11', 'S14')
('S3', 'S11', 'S15')
('S3', 'S11', 'S16')
('S3', 'S11', 'S17')
('S3', 'S11', 'S18')
('S3', 'S11', 'S19')
('S3', 'S11', 'S20')
('S3', 'S12', 'S13')
('S3', 'S12', 'S14')
('S3', 'S12', 'S15')
('S3', 'S12', 'S16')
('S3', 'S12', 'S17')
('S3', 'S12', 'S18')
('S3', 'S12', 'S19')
('S3', 'S12', 'S20')
('S3', 'S13', 'S14')
('S3', 'S13', 'S15')
('S3', 'S13', 'S16')
('S3', 'S13', 'S17')
('S3', 'S13', 'S18')
('S3', 'S13', 'S19')
('S3', 'S13', 'S20')
('S3', 'S14', 'S15')
('S3', 'S14', 'S16')
('S3', 'S14', 'S17')
('S3', 'S14', 'S18')
('S3', 'S14', 'S19')
('S3', 'S14', 'S20')
('S3', 'S15', 'S16')
('S3', 'S15', 'S17')
('S3', 'S15', 'S18')
('S3', 'S15', 'S19')
('S3', 'S15', 'S20')
('S3', 'S16', 'S17')
('S3', 'S16', 'S18')
('S3', 'S16', 'S19')
('S3', 'S16', 'S20')
('S3', 'S17', 'S18')
('S3', 'S17', 'S19')
('S3', 'S17', 'S20')
('S3', 'S18', 'S19')
('S3', 'S18', 'S20')
('S3', 'S19', 'S20')
('S4', 'S5', 'S6')
('S4', 'S5', 'S

('S9', 'S10', 'S12')
('S9', 'S10', 'S13')
('S9', 'S10', 'S14')
('S9', 'S10', 'S15')
('S9', 'S10', 'S16')
('S9', 'S10', 'S17')
('S9', 'S10', 'S18')
('S9', 'S10', 'S19')
('S9', 'S10', 'S20')
('S9', 'S11', 'S12')
('S9', 'S11', 'S13')
('S9', 'S11', 'S14')
('S9', 'S11', 'S15')
('S9', 'S11', 'S16')
('S9', 'S11', 'S17')
('S9', 'S11', 'S18')
('S9', 'S11', 'S19')
('S9', 'S11', 'S20')
('S9', 'S12', 'S13')
('S9', 'S12', 'S14')
('S9', 'S12', 'S15')
('S9', 'S12', 'S16')
('S9', 'S12', 'S17')
('S9', 'S12', 'S18')
('S9', 'S12', 'S19')
('S9', 'S12', 'S20')
('S9', 'S13', 'S14')
('S9', 'S13', 'S15')
('S9', 'S13', 'S16')
('S9', 'S13', 'S17')
('S9', 'S13', 'S18')
('S9', 'S13', 'S19')
('S9', 'S13', 'S20')
('S9', 'S14', 'S15')
('S9', 'S14', 'S16')
('S9', 'S14', 'S17')
('S9', 'S14', 'S18')
('S9', 'S14', 'S19')
('S9', 'S14', 'S20')
('S9', 'S15', 'S16')
('S9', 'S15', 'S17')
('S9', 'S15', 'S18')
('S9', 'S15', 'S19')
('S9', 'S15', 'S20')
('S9', 'S16', 'S17')
('S9', 'S16', 'S18')
('S9', 'S16', 'S19')
('S9', 'S16',

('S1', 'S3', 'S4', 'S18')
('S1', 'S3', 'S4', 'S19')
('S1', 'S3', 'S4', 'S20')
('S1', 'S3', 'S5', 'S6')
('S1', 'S3', 'S5', 'S7')
('S1', 'S3', 'S5', 'S8')
('S1', 'S3', 'S5', 'S9')
('S1', 'S3', 'S5', 'S10')
('S1', 'S3', 'S5', 'S11')
('S1', 'S3', 'S5', 'S12')
('S1', 'S3', 'S5', 'S13')
('S1', 'S3', 'S5', 'S14')
('S1', 'S3', 'S5', 'S15')
('S1', 'S3', 'S5', 'S16')
('S1', 'S3', 'S5', 'S17')
('S1', 'S3', 'S5', 'S18')
('S1', 'S3', 'S5', 'S19')
('S1', 'S3', 'S5', 'S20')
('S1', 'S3', 'S6', 'S7')
('S1', 'S3', 'S6', 'S8')
('S1', 'S3', 'S6', 'S9')
('S1', 'S3', 'S6', 'S10')
('S1', 'S3', 'S6', 'S11')
('S1', 'S3', 'S6', 'S12')
('S1', 'S3', 'S6', 'S13')
('S1', 'S3', 'S6', 'S14')
('S1', 'S3', 'S6', 'S15')
('S1', 'S3', 'S6', 'S16')
('S1', 'S3', 'S6', 'S17')
('S1', 'S3', 'S6', 'S18')
('S1', 'S3', 'S6', 'S19')
('S1', 'S3', 'S6', 'S20')
('S1', 'S3', 'S7', 'S8')
('S1', 'S3', 'S7', 'S9')
('S1', 'S3', 'S7', 'S10')
('S1', 'S3', 'S7', 'S11')
('S1', 'S3', 'S7', 'S12')
('S1', 'S3', 'S7', 'S13')
('S1', 'S3', 'S7', 'S

('S1', 'S5', 'S16', 'S20')
('S1', 'S5', 'S17', 'S18')
('S1', 'S5', 'S17', 'S19')
('S1', 'S5', 'S17', 'S20')
('S1', 'S5', 'S18', 'S19')
('S1', 'S5', 'S18', 'S20')
('S1', 'S5', 'S19', 'S20')
('S1', 'S6', 'S7', 'S8')
('S1', 'S6', 'S7', 'S9')
('S1', 'S6', 'S7', 'S10')
('S1', 'S6', 'S7', 'S11')
('S1', 'S6', 'S7', 'S12')
('S1', 'S6', 'S7', 'S13')
('S1', 'S6', 'S7', 'S14')
('S1', 'S6', 'S7', 'S15')
('S1', 'S6', 'S7', 'S16')
('S1', 'S6', 'S7', 'S17')
('S1', 'S6', 'S7', 'S18')
('S1', 'S6', 'S7', 'S19')
('S1', 'S6', 'S7', 'S20')
('S1', 'S6', 'S8', 'S9')
('S1', 'S6', 'S8', 'S10')
('S1', 'S6', 'S8', 'S11')
('S1', 'S6', 'S8', 'S12')
('S1', 'S6', 'S8', 'S13')
('S1', 'S6', 'S8', 'S14')
('S1', 'S6', 'S8', 'S15')
('S1', 'S6', 'S8', 'S16')
('S1', 'S6', 'S8', 'S17')
('S1', 'S6', 'S8', 'S18')
('S1', 'S6', 'S8', 'S19')
('S1', 'S6', 'S8', 'S20')
('S1', 'S6', 'S9', 'S10')
('S1', 'S6', 'S9', 'S11')
('S1', 'S6', 'S9', 'S12')
('S1', 'S6', 'S9', 'S13')
('S1', 'S6', 'S9', 'S14')
('S1', 'S6', 'S9', 'S15')
('S1', '

...

('S4', 'S12', 'S14', 'S15')
('S4', 'S12', 'S14', 'S17')
('S4', 'S12', 'S14', 'S18')
('S4', 'S12', 'S14', 'S19')
('S4', 'S12', 'S14', 'S20')
('S4', 'S12', 'S15', 'S17')
('S4', 'S12', 'S15', 'S18')
('S4', 'S12', 'S15', 'S19')
('S4', 'S12', 'S15', 'S20')
('S4', 'S12', 'S16', 'S18')
('S4', 'S12', 'S16', 'S19')
('S4', 'S12', 'S16', 'S20')
('S4', 'S12', 'S17', 'S18')
('S4', 'S12', 'S17', 'S19')
('S4', 'S12', 'S17', 'S20')
('S4', 'S12', 'S18', 'S19')
('S4', 'S12', 'S18', 'S20')
('S4', 'S12', 'S19', 'S20')
('S4', 'S13', 'S14', 'S15')
('S4', 'S13', 'S14', 'S17')
('S4', 'S13', 'S14', 'S18')
('S4', 'S13', 'S14', 'S19')
('S4', 'S13', 'S14', 'S20')
('S4', 'S13', 'S15', 'S17')
('S4', 'S13', 'S15', 'S18')
('S4', 'S13', 'S15', 'S19')
('S4', 'S13', 'S15', 'S20')
('S4', 'S13', 'S16', 'S18')
('S4', 'S13', 'S16', 'S19')
('S4', 'S13', 'S16', 'S20')
('S4', 'S13', 'S17', 'S18')
('S4', 'S13', 'S17', 'S19')
('S4', 'S13', 'S17', 'S20')
('S4', 'S13', 'S18', 'S19')
('S4', 'S13', 'S18', 'S20')
('S4', 'S13', 'S19',

('S5', 'S9', 'S11', 'S18')
('S5', 'S9', 'S11', 'S19')
('S5', 'S9', 'S11', 'S20')
('S5', 'S9', 'S12', 'S13')
('S5', 'S9', 'S12', 'S14')
('S5', 'S9', 'S12', 'S15')
('S5', 'S9', 'S12', 'S17')
('S5', 'S9', 'S12', 'S18')
('S5', 'S9', 'S12', 'S19')
('S5', 'S9', 'S12', 'S20')
('S5', 'S9', 'S13', 'S14')
('S5', 'S9', 'S13', 'S15')
('S5', 'S9', 'S13', 'S17')
('S5', 'S9', 'S13', 'S18')
('S5', 'S9', 'S13', 'S19')
('S5', 'S9', 'S13', 'S20')
('S5', 'S9', 'S14', 'S15')
('S5', 'S9', 'S14', 'S17')
('S5', 'S9', 'S14', 'S18')
('S5', 'S9', 'S14', 'S19')
('S5', 'S9', 'S14', 'S20')
('S5', 'S9', 'S15', 'S16')
('S5', 'S9', 'S15', 'S17')
('S5', 'S9', 'S15', 'S18')
('S5', 'S9', 'S15', 'S19')
('S5', 'S9', 'S15', 'S20')
('S5', 'S9', 'S16', 'S18')
('S5', 'S9', 'S16', 'S19')
('S5', 'S9', 'S16', 'S20')
('S5', 'S9', 'S17', 'S18')
('S5', 'S9', 'S17', 'S19')
('S5', 'S9', 'S17', 'S20')
('S5', 'S9', 'S18', 'S19')
('S5', 'S9', 'S18', 'S20')
('S5', 'S9', 'S19', 'S20')
('S5', 'S10', 'S11', 'S12')
('S5', 'S10', 'S11', 'S13')

('S6', 'S9', 'S10', 'S17')
('S6', 'S9', 'S10', 'S18')
('S6', 'S9', 'S10', 'S19')
('S6', 'S9', 'S10', 'S20')
('S6', 'S9', 'S11', 'S12')
('S6', 'S9', 'S11', 'S13')
('S6', 'S9', 'S11', 'S14')
('S6', 'S9', 'S11', 'S15')
('S6', 'S9', 'S11', 'S17')
('S6', 'S9', 'S11', 'S18')
('S6', 'S9', 'S11', 'S19')
('S6', 'S9', 'S11', 'S20')
('S6', 'S9', 'S12', 'S15')
('S6', 'S9', 'S12', 'S18')
('S6', 'S9', 'S12', 'S19')
('S6', 'S9', 'S12', 'S20')
('S6', 'S9', 'S13', 'S15')
('S6', 'S9', 'S13', 'S18')
('S6', 'S9', 'S13', 'S19')
('S6', 'S9', 'S13', 'S20')
('S6', 'S9', 'S14', 'S15')
('S6', 'S9', 'S14', 'S18')
('S6', 'S9', 'S14', 'S19')
('S6', 'S9', 'S14', 'S20')
('S6', 'S9', 'S15', 'S17')
('S6', 'S9', 'S15', 'S18')
('S6', 'S9', 'S15', 'S19')
('S6', 'S9', 'S15', 'S20')
('S6', 'S9', 'S16', 'S18')
('S6', 'S9', 'S16', 'S19')
('S6', 'S9', 'S16', 'S20')
('S6', 'S9', 'S17', 'S18')
('S6', 'S9', 'S17', 'S19')
('S6', 'S9', 'S17', 'S20')
('S6', 'S9', 'S18', 'S19')
('S6', 'S9', 'S18', 'S20')
('S6', 'S9', 'S19', 'S20')
(

('S7', 'S11', 'S16', 'S18')
('S7', 'S11', 'S16', 'S19')
('S7', 'S11', 'S16', 'S20')
('S7', 'S11', 'S17', 'S18')
('S7', 'S11', 'S17', 'S19')
('S7', 'S11', 'S17', 'S20')
('S7', 'S11', 'S18', 'S19')
('S7', 'S11', 'S18', 'S20')
('S7', 'S11', 'S19', 'S20')
('S7', 'S12', 'S13', 'S18')
('S7', 'S12', 'S13', 'S19')
('S7', 'S12', 'S13', 'S20')
('S7', 'S12', 'S14', 'S18')
('S7', 'S12', 'S14', 'S19')
('S7', 'S12', 'S14', 'S20')
('S7', 'S12', 'S15', 'S18')
('S7', 'S12', 'S15', 'S19')
('S7', 'S12', 'S15', 'S20')
('S7', 'S12', 'S16', 'S19')
('S7', 'S12', 'S17', 'S18')
('S7', 'S12', 'S17', 'S19')
('S7', 'S12', 'S17', 'S20')
('S7', 'S12', 'S18', 'S19')
('S7', 'S12', 'S18', 'S20')
('S7', 'S12', 'S19', 'S20')
('S7', 'S13', 'S14', 'S18')
('S7', 'S13', 'S14', 'S19')
('S7', 'S13', 'S14', 'S20')
('S7', 'S13', 'S15', 'S18')
('S7', 'S13', 'S15', 'S19')
('S7', 'S13', 'S15', 'S20')
('S7', 'S13', 'S16', 'S19')
('S7', 'S13', 'S17', 'S18')
('S7', 'S13', 'S17', 'S19')
('S7', 'S13', 'S17', 'S20')
('S7', 'S13', 'S18',

('S10', 'S11', 'S12', 'S15')
('S10', 'S11', 'S12', 'S17')
('S10', 'S11', 'S12', 'S18')
('S10', 'S11', 'S12', 'S19')
('S10', 'S11', 'S12', 'S20')
('S10', 'S11', 'S13', 'S14')
('S10', 'S11', 'S13', 'S15')
('S10', 'S11', 'S13', 'S17')
('S10', 'S11', 'S13', 'S18')
('S10', 'S11', 'S13', 'S19')
('S10', 'S11', 'S13', 'S20')
('S10', 'S11', 'S14', 'S15')
('S10', 'S11', 'S14', 'S17')
('S10', 'S11', 'S14', 'S18')
('S10', 'S11', 'S14', 'S19')
('S10', 'S11', 'S14', 'S20')
('S10', 'S11', 'S15', 'S17')
('S10', 'S11', 'S15', 'S18')
('S10', 'S11', 'S15', 'S19')
('S10', 'S11', 'S15', 'S20')
('S10', 'S11', 'S16', 'S18')
('S10', 'S11', 'S16', 'S19')
('S10', 'S11', 'S16', 'S20')
('S10', 'S11', 'S17', 'S18')
('S10', 'S11', 'S17', 'S19')
('S10', 'S11', 'S17', 'S20')
('S10', 'S11', 'S18', 'S19')
('S10', 'S11', 'S18', 'S20')
('S10', 'S11', 'S19', 'S20')
('S10', 'S12', 'S13', 'S18')
('S10', 'S12', 'S13', 'S19')
('S10', 'S12', 'S13', 'S20')
('S10', 'S12', 'S14', 'S15')
('S10', 'S12', 'S14', 'S18')
('S10', 'S12',

In [18]:
list_of_sets = pd.DataFrame()

if __name__ == "__main__":
    shops = vol_av_daily_per_month.index #all the shop names
    
    for r in range(0, 5): #the max amount of shops per set is 4
        for i in range(0,len(rSubset(shops, r))):
            vol_sum = 0
            for j in range(0,r):
                vol_sum = vol_sum + vol_av_daily_per_month[1][rSubset(shops, r)[i][j]] #value of shop 
        
            if vol_sum <= 20:
                temp = pd.DataFrame(
                    {
                        'subset': [rSubset(shops, r)[i]]
                    }
                )
                list_of_sets = pd.concat([list_of_sets, temp])

In [23]:
list_of_sets

subset
0                     ()
0                  (S1,)
0                  (S2,)
0                  (S3,)
0                  (S4,)
..                   ...
0   (S16, S17, S18, S19)
0   (S16, S17, S18, S20)
0   (S16, S17, S19, S20)
0   (S16, S18, S19, S20)
0   (S17, S18, S19, S20)

[5804 rows x 1 columns]

In [20]:
def rSubset(subsets, r):
  
    # return list of all subsets of length r
    # to deal with duplicate subsets use 
    # set(list(combinations(shops, r)))
    return list(combinations(subsets, r))
  
# Driver Function
if __name__ == "__main__":
    subsets = list_of_sets['subset'].values #all the shop names
    r = 2
    print(rSubset(subsets, r))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

